<a href="https://colab.research.google.com/github/tarun42/Flipr-Ml-Gryf1ndor/blob/master/Flipr_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pmdarima

     |████████████████████████████████| 1.5MB 8.0MB/s 
     |████████████████████████████████| 8.7MB 31.7MB/s 
     |████████████████████████████████| 2.1MB 37.3MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import math
from sklearn.preprocessing import LabelEncoder 
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
%matplotlib inline 


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
from google.colab import files
uploaded = files.upload() 

Saving Test_dataset.xlsx to Test_dataset.xlsx


In [224]:
df = pd.read_excel("Test_dataset.xlsx", sheet_name=1,)
df=df.drop(0)
df

,City,Time Series Data,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
1,Tuensang,1614,1946,2372,2500,2769
2,Lakshmeshwar,369179,445559,541045,572860,636502
3,Zira,140581,169665,206030,218138,242367
4,Yawal,2557179,3086249,3747597,3968032,4408916
5,Thana Bhawan,1800363,2172850,2638469,2793662,3104060
...,...,...,...,...,...,...
497,Nandaprayag,61420,74125,90018,95302,105882
498,Kirtinagar,61420,74125,90018,95302,105882
499,Kedarnath,61420,74125,90018,95302,105882
500,Gangotri,61420,74125,90018,95302,105882


In [225]:
df.columns=['City', 'April', 'May', 'June', 'July', 'August']
indexs=df.iloc[0: ,0]
df.index=indexs
# df=df.drop(0,axis=0)
# df=df.dropna()
# df=df.drop(['NaN'],axis=0)
print(df)

                      City    April      May     June     July   August
City                                                                   
Tuensang          Tuensang     1614     1946     2372     2500     2769
Lakshmeshwar  Lakshmeshwar   369179   445559   541045   572860   636502
Zira                  Zira   140581   169665   206030   218138   242367
Yawal                Yawal  2557179  3086249  3747597  3968032  4408916
Thana Bhawan  Thana Bhawan  1800363  2172850  2638469  2793662  3104060
...                    ...      ...      ...      ...      ...      ...
Nandaprayag    Nandaprayag    61420    74125    90018    95302   105882
Kirtinagar      Kirtinagar    61420    74125    90018    95302   105882
Kedarnath        Kedarnath    61420    74125    90018    95302   105882
Gangotri          Gangotri    61420    74125    90018    95302   105882
Kumarganj        Kumarganj  1800363  2172850  2638469  2793662  3104060

[501 rows x 6 columns]


In [226]:
missing_val = (df.isnull().sum())
missing_val

City       0
April     55
May       57
June      32
July      17
August     0
dtype: int64

In [227]:
def miss(row):
  for i in range(2,6):
    diff = []
    if( (not math.isnan(row[i-1]))  and (not math.isnan(row[i]))):     
      diff.append(row[i] - row[i-1])
      
    # if(len(diff)>0 and len(diff)<4):
    #   print(row)
  
  if(len(diff)>0):
    diff = sum(diff)/len(diff)
    if(math.isnan(row[5]) and  (not math.isnan(row[4]))):     
      row[5] = row[4] + diff
    for i in range(1,5):
      if(math.isnan(row[i]) and (not math.isnan(row[i+1]))):
        
        row[i] = row[i+1] - diff
  return row

In [228]:
def miss2(row):
  for i in range(2,6):
    diff = []
    if( (not math.isnan(row[i-1]))  and (not math.isnan(row[i]))):     
      diff.append(row[i] - row[i-1])
    
  if(len(diff)>0):
    diff = sum(diff)/len(diff)
    if(math.isnan(row[5]) and  (not math.isnan(row[4]))):  
      
      row[5] = row[4] + diff
    for i in range(4,0,-1):
      if(math.isnan(row[i]) and (not math.isnan(row[i+1]))):
        
        row[i] = row[i+1] - diff
    
  return row

In [229]:
df = df.apply( miss , axis = 1)
df.head(10)
missing_val = (df.isnull().sum())
missing_val

City       0
April     36
May       14
June      17
July      17
August     0
dtype: int64

In [230]:
df = df.apply( miss2 , axis = 1)
df.head(10)
missing_val = (df.isnull().sum())
missing_val

City       0
April      3
May        7
June      17
July      17
August     0
dtype: int64

In [231]:
sum(df.apply(lambda x:sum(x.isnull().values) , axis = 1) > 0)

17

In [245]:
def miss3(row):
  c=0
  for i in range(1,6):
    if(math.isnan(row[i])):
      c=c+1
  if(c==3):
    row[3]=(row[1]+row[-1])/2
    row[2]=(row[1]+row[3])/2
    row[4]=(row[3]+row[-1])/2
  if(c==2):
    row[3]=(row[1]+row[-1])/2
    row[4]=(row[3]+row[5])/2
  if(c==4):
    if(row[0]=='Dharchula'):
      row[1]=61420
      row[2]=74125
      row[3]=90018
      row[4]=95302
    if(row[0]=='Adyar'):
      row[1]=369179
      row[2]=445559
      row[3]=541045
      row[4]=572860
    if(row[0]=='Narsinghgarh'):
      row[1]=224440
      row[2]=294964
      row[3]=38178
      row[4]=379239
  else: return row
  return row

In [246]:
df = df.apply( miss3 , axis = 1)
df.head(10)
missing_val = (df.isnull().sum())
missing_val

City      0
April     0
May       0
June      0
July      0
August    0
dtype: int64

In [247]:
df.isnull().sum()

City      0
April     0
May       0
June      0
July      0
August    0
dtype: int64

In [248]:
y=0
for i in range(len(df.values)):
  # print(df.values[i][0])
  if( math.isnan(df.values[i][1])):
    y=y+1
  if( math.isnan(df.values[i][2])):
    y=y+1
  if( math.isnan(df.values[i][3])):
    y=y+1
  if( math.isnan(df.values[i][4])):
    y=y+1
  if( math.isnan(df.values[i][5])):
    y=y+1
  # print(i)
  if(y>=1):
    print(i,df.values[i])
    
  y=0



In [ ]:
df

In [ ]:
print(indexs)

In [249]:
df=df.drop(['City'],axis=1)
print(df)

                  April        May       June       July   August
City                                                             
Tuensang         1614.0     1946.0     2372.0     2500.0     2769
Lakshmeshwar   369179.0   445559.0   541045.0   572860.0   636502
Zira           140581.0   169665.0   206030.0   218138.0   242367
Yawal         2557179.0  3086249.0  3747597.0  3968032.0  4408916
Thana Bhawan  1800363.0  2172850.0  2638469.0  2793662.0  3104060
...                 ...        ...        ...        ...      ...
Nandaprayag     61420.0    74125.0    90018.0    95302.0   105882
Kirtinagar      61420.0    74125.0    90018.0    95302.0   105882
Kedarnath       61420.0    74125.0    90018.0    95302.0   105882
Gangotri        61420.0    74125.0    90018.0    95302.0   105882
Kumarganj     1800363.0  2172850.0  2638469.0  2793662.0  3104060

[501 rows x 5 columns]


In [250]:
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")

In [251]:
print(df.isnull().sum())

April     0
May       0
June      0
July      0
August    0
dtype: int64


In [252]:
df1=df.T
print(df1)
# df2=df1.iloc[1:]
df2=df1
address = ['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01','2020-05-01'] 
df2['Date'] = address 
# df2.plot.hist(subplots=True)
print(df2)
df2.to_csv('df2.csv')

City    Tuensang  Lakshmeshwar      Zira  ...  Kedarnath  Gangotri  Kumarganj
April     1614.0      369179.0  140581.0  ...    61420.0   61420.0  1800363.0
May       1946.0      445559.0  169665.0  ...    74125.0   74125.0  2172850.0
June      2372.0      541045.0  206030.0  ...    90018.0   90018.0  2638469.0
July      2500.0      572860.0  218138.0  ...    95302.0   95302.0  2793662.0
August    2769.0      636502.0  242367.0  ...   105882.0  105882.0  3104060.0

[5 rows x 501 columns]
City    Tuensang  Lakshmeshwar      Zira  ...  Gangotri  Kumarganj        Date
April     1614.0      369179.0  140581.0  ...   61420.0  1800363.0  2020-01-01
May       1946.0      445559.0  169665.0  ...   74125.0  2172850.0  2020-02-01
June      2372.0      541045.0  206030.0  ...   90018.0  2638469.0  2020-03-01
July      2500.0      572860.0  218138.0  ...   95302.0  2793662.0  2020-04-01
August    2769.0      636502.0  242367.0  ...  105882.0  3104060.0  2020-05-01

[5 rows x 502 columns]


In [253]:
df3 = pd.read_csv("df2.csv",index_col='Date',parse_dates=True)
df3.drop(['Unnamed: 0'],axis = 1,inplace=True)
print(df3.head())

            Tuensang  Lakshmeshwar      Zira  ...  Kedarnath  Gangotri  Kumarganj
Date                                          ...                                
2020-01-01    1614.0      369179.0  140581.0  ...    61420.0   61420.0  1800363.0
2020-02-01    1946.0      445559.0  169665.0  ...    74125.0   74125.0  2172850.0
2020-03-01    2372.0      541045.0  206030.0  ...    90018.0   90018.0  2638469.0
2020-04-01    2500.0      572860.0  218138.0  ...    95302.0   95302.0  2793662.0
2020-05-01    2769.0      636502.0  242367.0  ...   105882.0  105882.0  3104060.0

[5 rows x 501 columns]


In [254]:
def addtest(dataset):
  result = adfuller(dataset)
  print('ADF Statistic: {}'.format(result[0]))
  print('p-value: {}'.format(result[1]))
  print('Critical Values:')
  for key, value in result[4].items():
    print('\t{}: {}'.format(key, value))

In [255]:
for i in df3.columns:
  addtest(df3[str(i)])

ADF Statistic: -1.1061240511744446
p-value: 0.7126235353697294
Critical Values:
	1%: -7.355440625
	5%: -4.474365000000001
	10%: -3.1269325
ADF Statistic: -1.065028600600847
p-value: 0.7288147744148463
Critical Values:
	1%: -7.355440625
	5%: -4.474365000000001
	10%: -3.1269325
ADF Statistic: -1.065336694639422
p-value: 0.7286954917621133
Critical Values:
	1%: -7.355440625
	5%: -4.474365000000001
	10%: -3.1269325
ADF Statistic: -1.0648560344614606
p-value: 0.7288815716433258
Critical Values:
	1%: -7.355440625
	5%: -4.474365000000001
	10%: -3.1269325
ADF Statistic: -1.0648690330758963
p-value: 0.728876540465583
Critical Values:
	1%: -7.355440625
	5%: -4.474365000000001
	10%: -3.1269325
ADF Statistic: -1.065028600600847
p-value: 0.7288147744148463
Critical Values:
	1%: -7.355440625
	5%: -4.474365000000001
	10%: -3.1269325
ADF Statistic: -0.7022571198702038
p-value: 0.8461640263675436
Critical Values:
	1%: -7.355440625
	5%: -4.474365000000001
	10%: -3.1269325
ADF Statistic: -0.7025691821316

In [256]:
stepwise_fit=auto_arima(df3['Tuensang'],trace=True,suppress_warnings=True)
stepwise_fit.df_model()

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=156.799, Time=0.14 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=78.367, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=78.191, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=93.498, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.21 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.568 seconds


3

In [257]:
print(df3.shape)
train=df3.iloc[0:4]
test=df3.iloc[4:]
print(train.shape,test.shape)

(5, 501)
(4, 501) (1, 501)


In [258]:
model_list=[]
for i  in df3.columns:
  model=ARIMA(train[str(i)],order=(1,0,0))
  model=model.fit()
  model_list.append(model)



In [ ]:
pred_list=[]
start=len(train)
end=len(train)+len(test)-1
for m in model_list:
  pred=m.predict(start=start,end=end,typ='levels')
  pred.index=df3.index[start:end+1]
  pred_list.append(pred)


In [260]:
print(pred_list)

[Date
2020-05-01    2333.924711
dtype: float64, Date
2020-05-01    534949.027856
dtype: float64, Date
2020-05-01    203786.416632
dtype: float64, Date
2020-05-01    4.006310e+06
dtype: float64, Date
2020-05-01    2.820417e+06
dtype: float64, Date
2020-05-01    534949.027856
dtype: float64, Date
2020-05-01    3.714109e+06
dtype: float64, Date
2020-05-01    106358.394824
dtype: float64, Date
2020-05-01    537181.784343
dtype: float64, Date
2020-05-01    1.334620e+06
dtype: float64, Date
2020-05-01    1.251803e+06
dtype: float64, Date
2020-05-01    4.006310e+06
dtype: float64, Date
2020-05-01    4767.636369
dtype: float64, Date
2020-05-01    141048.669066
dtype: float64, Date
2020-05-01    4.006310e+06
dtype: float64, Date
2020-05-01    534949.027856
dtype: float64, Date
2020-05-01    534949.027856
dtype: float64, Date
2020-05-01    354213.959286
dtype: float64, Date
2020-05-01    351234.965285
dtype: float64, Date
2020-05-01    199895.07629
dtype: float64, Date
2020-05-01    537181.78434

In [ ]:
# from sklearn.metrics import mean_squared_error
# from math import sqrt
# c=0
# for i in df3.columns:
#   rms = sqrt(mean_squared_error(pred_list[c], test[str(i)]))
#   print(rms)
#   c=c+1

In [262]:
model2_list=[]
for i in df3.columns:
  model2=ARIMA(df3[str(i)],order=(1,0,0))
  model2=model2.fit()
  model2_list.append(model2)
# model.summary()
df3.tail()

,Tuensang,Lakshmeshwar,Zira,Yawal,Thana Bhawan,Ramdurg,Pulgaon,Sadasivpet,Nargund,Neem-Ka-Thana,Memari,Nilanga,Naharlagun,Pakaur,Wai,Tarikere,Malavalli,Raisen,Lahar,Uravakonda,Savanur,Sirohi,Udhampur,Umarga,Pratapgarh,Lingsugur,Usilampatti,Palia Kalan,Wokha,Rajpipla,Vijayapura,Rawatbhata,Sangaria,Paithan,Rahuri,Patti,Zaidpur,Lalsot,Maihar,Degana,...,Sultanpur,Herbertpur,Rudraprayag,Niwai,Bageshwar,Gochar,Doiwala,Tarbha,Karnaprayag,Sanivarapupeta,Lohaghat,Bhimtal,Lalkuan,Kaladhungi,Naura,Mahua Dabra Haripura,Chandpara,Dharchula,Adyar,Narsinghgarh.1,Barkot,Didihat,Satrampadu,Warud,Asarganj,Bhowali,Shaktigarh,Narendranagar,Champawat,Sarsod,Bhaiseena,Dwarahat,Badrinath,Dogadda,Devprayag,Nandaprayag,Kirtinagar,Kedarnath,Gangotri,Kumarganj
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,1614.0,369179.0,140581.0,2557179.0,1800363.0,369179.0,2645365.0,75663.0,381917.0,855688.0,863918.0,2557179.0,3317.0,97323.0,2557179.0,369179.0,369179.0,244400.0,244400.00,137963.0,381917.0,855688.0,33977.0,2557179.0,855688.0,369179.0,2717138.0,1800363.0,1614.0,185267.0,413761.0,958986.0,855688.0,2557179.0,2557179.0,140581.0,1800363.0,855688.0,244400.0,855688.0,...,61420.0,61420.0,61420.0,855688.0,61420.0,61420.0,61420.0,38851.0,61420.0,137963.0,61420.0,61420.0,61420.0,61420.0,140581.0,61420.0,863918.0,61420.0,369179.0,224440.0,61420.0,61420.0,137963.0,2557179.0,535775.0,61420.0,61420.0,61420.0,61420.0,175816.0,855688.0,61420.0,61420.0,61420.0,61420.0,61420.0,61420.0,61420.0,61420.0,1800363.0
2020-02-01,1946.0,445559.0,169665.0,3086249.0,2172850.0,445559.0,3086249.0,88263.0,445559.0,1032726.0,1042658.0,3086249.0,4305.0,117458.0,3086249.0,445559.0,445559.0,294964.0,294964.00,166506.0,445559.0,1032726.0,41006.0,3086249.0,1032726.0,445559.0,3279303.0,2172850.0,1946.0,213756.0,477403.0,1106509.0,1032726.0,3086249.0,3086249.0,169665.0,2172850.0,1032726.0,294964.0,1032726.0,...,74125.0,74125.0,74125.0,1032726.0,74125.0,74125.0,74125.0,46888.0,74125.0,166506.0,74125.0,74125.0,74125.0,74125.0,169665.0,74125.0,1042658.0,74125.0,445559.0,294964.0,74125.0,74125.0,166506.0,3086249.0,646624.0,74125.0,74125.0,74125.0,74125.0,212191.0,1032726.0,74125.0,74125.0,74125.0,74125.0,74125.0,74125.0,74125.0,74125.0,2172850.0
2020-03-01,2372.0,541045.0,206030.0,3747597.0,2638469.0,541045.0,3747597.0,107184.0,541045.0,1180265.0,1266093.0,3747597.0,4867.0,142635.0,3747597.0,541045.0,541045.0,358178.0,332882.50,202194.0,541045.0,1254032.0,49801.0,3747597.0,1254032.0,541045.0,3982019.0,2638469.0,2372.0,242245.0,541045.0,1254032.0,1254032.0,3747597.0,3747597.0,206030.0,2638469.0,1254032.0,358178.0,1254032.0,...,90018.0,90018.0,90018.0,1254032.0,90018.0,90018.0,90018.0,56943.0,90018.0,202194.0,90018.0,90018.0,90018.0,90018.0,206030.0,90018.0,1266093.0,90018.0,541045.0,38178.0,90018.0,90018.0,202194.0,3747597.0,785194.0,90018.0,90018.0,90018.0,90018.0,257668.0,1254032.0,90018.0,90018.0,90018.0,90018.0,90018.0,90018.0,90018.0,90018.0,2638469.0
2020-04-01,2500.0,572860.0,218138.0,3968032.0,2793662.0,572860.0,3968032.0,113478.0,572860.0,1327788.0,1340558.0,3968032.0,5143.0,151015.0,3968032.0,572860.0,572860.0,379237.0,377123.75,214077.0,572860.0,1327788.0,52719.0,3968032.0,1327788.0,572860.0,4216244.0,2793662.0,2500.0,256484.0,572860.0,1327788.0,1327788.0,3968032.0,3968032.0,218138.0,2793662.0,1327788.0,379237.0,1327788.0,...,95302.0,95302.0,95302.0,1327788.0,95302.0,95302.0,95302.0,60282.0,95302.0,214077.0,95302.0,95302.0,95302.0,95302.0,218138.0,95302.0,1340558.0,95302.0,572860.0,379239.0,95302.0,95302.0,214077.0,3968032.0,831371.0,95302.0,95302.0,95302.0,95302.0,272814.0,1327788.0,95302.0,95302.0,95302.0,95302.0,95302.0,95302.0,95302.0,95302.0,2793662.0
2020-05-01,2769.0,636502.0,242367.0,4408916.0,3104060.0,636502.0,4408916.0,126078.0,636502.0,1475311.0,1489500.0,4408916.0,5705.0,167785.0,4408916.0,636502.0,636502.0,421365.0,421365.00,237854.0,636502.0,1475311.0,58568.0,4408916.0,1475311.0,636502.0,4684707.0,3104060.0,2769.0,284973.

In [265]:
pred2_list=[]
city_list=[]
start=len(df3)
end=len(train)+len(test)-1
c=0
for m2 in model2_list:
  pred2=m2.predict(start=start,end=start,typ='levels').rename('ARIMA pred all')
  pred2_list.append(pred2)
  city_list.append(df3.columns[c])
  c=c+1
# pred.index=df3.index[start:end+1]
for i in range(len(pred2_list)):
  print(pred2_list[i][0])
  print(city_list[i])


2647.994635505656
Tuensang
640376.5551586517
Lakshmeshwar
231905.21497440402
Zira
4435383.327840765
Yawal
3123071.307522969
Thana Bhawan
640376.5551586517
Ramdurg
4429628.001155571
Pulgaon
120844.88161654997
Sadasivpet
610297.329669979
Nargund
1479325.1350785883
Neem-Ka-Thana
1498583.9941159235
Memari
4435383.327840765
Nilanga
5430.297163320506
Naharlagun
160572.0951527708
Pakaur
4435383.327840765
Wai
640376.5551586517
Tarikere
640376.5551586517
Malavalli
422791.9372259254
Raisen
422321.9106817764
Lahar
227592.71210261804
Uravakonda
610297.329669979
Savanur
1483191.9737001217
Sirohi
56027.46976418907
Udhampur
4435383.327840765
Umarga
1483191.9737001217
Pratapgarh
640376.5551586517
Lingsugur
4712612.535222533
Usilampatti
3123071.307522969
Palia Kalan
2647.994635505656
Wokha
274875.33790098544
Rajpipla
638954.8758549235
Vijayapura
1481130.2869473696
Rawatbhata
1483191.9737001217
Sangaria
4435383.327840765
Paithan
4435383.327840765
Rahuri
231905.21497440402
Patti
3123071.307522969
Zaidpur

In [268]:
fv=[]
for i in pred2_list:
  fv.append(int(i[0]))
print(len(fv))
print(len(city_list))



501
501


In [270]:
for i in fv:
  print(i,',')

2647 ,
640376 ,
231905 ,
4435383 ,
3123071 ,
640376 ,
4429628 ,
120844 ,
610297 ,
1479325 ,
1498583 ,
4435383 ,
5430 ,
160572 ,
4435383 ,
640376 ,
640376 ,
422791 ,
422321 ,
227592 ,
610297 ,
1483191 ,
56027 ,
4435383 ,
1483191 ,
640376 ,
4712612 ,
3123071 ,
2647 ,
274875 ,
638954 ,
1481130 ,
1483191 ,
4435383 ,
4435383 ,
231905 ,
3123071 ,
1483191 ,
422791 ,
1483191 ,
4712612 ,
4435383 ,
388541 ,
285548 ,
101309 ,
422791 ,
925756 ,
927164 ,
405451 ,
4425377 ,
161330 ,
161330 ,
978925 ,
64380 ,
285548 ,
227592 ,
3123071 ,
925756 ,
120632 ,
4712612 ,
1483191 ,
4435383 ,
303910 ,
925756 ,
982921 ,
3121397 ,
159820 ,
4436826 ,
3121397 ,
401506 ,
401506 ,
3121397 ,
3121397 ,
120632 ,
303910 ,
33375 ,
1483191 ,
4712612 ,
1483191 ,
227967 ,
4429628 ,
927089 ,
610297 ,
610297 ,
4496020 ,
272947 ,
272947 ,
120844 ,
3119985 ,
6127 ,
404080 ,
982892 ,
404080 ,
404080 ,
3123071 ,
4712612 ,
285548 ,
4712612 ,
3123071 ,
160572 ,
1483191 ,
4712612 ,
4435383 ,
3123071 ,
640376 ,
4435383 ,
6116 ,
6403

In [271]:
for i in city_list:
  print(i,',')

Tuensang ,
Lakshmeshwar ,
Zira ,
Yawal ,
Thana Bhawan ,
Ramdurg ,
Pulgaon ,
Sadasivpet ,
Nargund ,
Neem-Ka-Thana ,
Memari ,
Nilanga ,
Naharlagun ,
Pakaur ,
Wai ,
Tarikere ,
Malavalli ,
Raisen ,
Lahar ,
Uravakonda ,
Savanur ,
Sirohi ,
Udhampur ,
Umarga ,
Pratapgarh ,
Lingsugur ,
Usilampatti ,
Palia Kalan ,
Wokha ,
Rajpipla ,
Vijayapura ,
Rawatbhata ,
Sangaria ,
Paithan ,
Rahuri ,
Patti ,
Zaidpur ,
Lalsot ,
Maihar ,
Degana ,
Vedaranyam ,
Nawapur ,
Solan ,
Vapi ,
Almora ,
Sanawad ,
Warisaliganj ,
Revelganj ,
Sabalgarh ,
Tuljapur ,
Simdega ,
Musabani ,
Kodungallur ,
Phulabani ,
Umreth ,
Narsipatnam ,
Nautanwa ,
Rajgir ,
Yellandu ,
Sathyamangalam ,
Pilibanga ,
Morshi ,
Pehowa ,
Sonepur ,
Pappinisseri ,
Zamania ,
Mihijam ,
Purna ,
Shikarpur, Bulandshahr ,
Umaria ,
Porsa ,
Naugawan Sadat ,
Faizabad ,
Manuguru ,
Ellenabad ,
Udaipur ,
Pipar City ,
Nanjikottai ,
Taranagar ,
Yerraguntla ,
Satana ,
Sherghati ,
Sankeshwara ,
Madikeri ,
Thuraiyur ,
Sanand ,
Rajula ,
Kyathampalle ,
Shahabad, Rampur ,